<a href="https://colab.research.google.com/github/adithyakini/algo_intraday/blob/master/bhavcopy_nseindia2mysql.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install mysql.connector

     |████████████████████████████████| 11.9MB 8.6MB/s 
  Created wheel for mysql.connector: filename=mysql_connector-2.2.9-cp36-cp36m-linux_x86_64.whl size=247949 sha256=ed7917d275aa48e12d6ab08b97067802496fe2e6bc7e9b6ce1a9b1c6d7235b2a
  Stored in directory: /root/.cache/pip/wheels/8c/83/a1/f8b6d4bb1bd6208bbde1608bbfa7557504bed9eaf2ecf8c175
Successfully built mysql.connector


In [ ]:
import requests, zipfile, os, io, pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta
import mysql.connector
from mysql.connector import Error
import csv
import glob
import re
from progressbar import ProgressBar
from termcolor import colored, cprint

#https://honingds.com/blog/pandas-read_csv/#indexcol

#set the path to where the bhavcopies will be downloaded
base = '/content/drive/My Drive/algotrade/'
today = datetime.today().date()
dmonth={'01':'JAN','02':'FEB','03':'MAR','04':'APR','05':'MAY','06':'JUN','07':'JUL','08':'AUG','09':'SEP','10':'OCT','11':'NOV','12':'DEC'}
holiday = ['2020-04-02','2020-04-06','2020-04-10','2020-04-14','2020-05-01','2020-05-25','2020-10-02','2020-11-16','2020-11-30','2020-12-25']
# Before running this script , file called bhavcopy_date.txt need to be present in the "base" path.
# Opening file named bhavcopy_date.txt , it keeps track of the last downloaded date.
ltdl = open(base+'bhavcopy_date.txt','r')
lastdt = ltdl.read(10)
ltdl.close()
lastdt = datetime.strptime(lastdt,'%Y-%m-%d')
diff, wr = today-lastdt.date(), ''

for i in range(1,diff.days+1): #loop through all dates from the last date mentioned in the bhavcopy_date file until today.
  nextdt = lastdt+ relativedelta(days=i) #calculate the next day value
  #check if the date is a weekend or market holiday so that we can remove that from the loop , bhavcopies are not available for weekends.
  if (nextdt.weekday() == 5 or nextdt.weekday() == 6):
    cprint (nextdt.strftime('%Y-%m-%d')+' is a weekend','grey')
  elif nextdt.strftime('%Y-%m-%d') in holiday:
    cprint (nextdt.strftime('%Y-%m-%d')+' is a Market Holiday','grey')
  else:
    d, m, y = '%02d' % nextdt.day, '%02d' % nextdt.month, '%02d' % nextdt.year #extract day , month and year from  he date
    zpath = base+y+'/'+d+'.zip'
    if not os.path.isdir(base+y):#if there is no directory already present at the path with the year as a folder then create it
      os.mkdir(base+y)
      os.mkdir(base+y+'/Index')
      os.mkdir(base+y+'/Futures')
      os.mkdir(base+y+'/Forex')
    for i in range(3): #try to connect to the nseindia url to download the bhavcopy , 7 times , just incase website does not respond etc.
      while True:
        try:
          equities_bhavcopy=requests.get('https://archives.nseindia.com/content/historical/EQUITIES/'+y+'/'+dmonth[m]+'/cm'+d+dmonth[m]+y+'bhav.csv.zip')
        except requests.ConnectionError:
          print('No connection, retrying')
        break
    if equities_bhavcopy.status_code==200:#if the connection is successful
      dload=open(zpath, 'wb')
      dload.write(equities_bhavcopy.content)
      dload.close()
      #open the downlaoded bhavcopy and extract it
      z = zipfile.ZipFile(zpath, 'r')
      z.extractall(base+y+'/')
      z.close()
      os.remove(zpath)
      #reading and storing in 2 dictionaries because we need 2 columns from the MTO file deliverable and %deliverable which is not found in the bhavcopy.
      f, deldict = pd.read_csv(base+y+'/cm'+d+dmonth[m]+y+'bhav.csv'), {}  #reading the raw dl-ed bhav file
      f, deldict2 = pd.read_csv(base+y+'/cm'+d+dmonth[m]+y+'bhav.csv'), {}  #reading the raw dl-ed bhav file
      f = f[f['SERIES'] == 'EQ'] #retaining only EQ rows and leaving out bonds,options etc
      #cprint('cm'+d+dmonth[m]+y+'bhav.csv.zip download...SUCCESS', 'red', attrs=['blink'])
      print('cm'+d+dmonth[m]+y+'bhav.csv.zip download...SUCCESS')
      # a file called mto.dat holds deliverable data, it is useful data to know whether the delivery percentage of a stock has gone up , an indication that long term investors have gone up
      for i in range(7): #try to connect to the nseindia url to download the mto file , 7 times , just incase website does not respond etc.
        while True:
          try:
            deliverable = requests.get('https://archives.nseindia.com/archives/equities/mto/MTO_'+d+m+y+'.DAT').text.splitlines()
          except requests.ConnectionError:
            cprint('No connection, retrying','red')
          break
      print(''+d+m+y+'.DAT download ...SUCCESS')
      del deliverable[:4]
      for i in deliverable:
          c = i.split(',')
          if c[3] == 'EQ' :                
              deldict[c[2]] = c[5] #building delivarables dict
          if c[3] == 'EQ' :                
              deldict2[c[2]] = c[6] #building %delivarables dict
      dfdel = pd.DataFrame(list(deldict.items()), columns = ['SYMBOL', 'DELIVERABLE'])
      dfdel2 = pd.DataFrame(list(deldict2.items()), columns = ['SYMBOL', '%DELIVERABLE'])
      f = f.merge(dfdel, on='SYMBOL', how='left')      #left merge of delivarables here
      f = f.merge(dfdel2, on='SYMBOL', how='left')      #left merge of delivarables here
      #write a new csv, bhavcopydate as a column in the csv and get rid of the downloaded file
      f['BHAVCOPYDATE'] = pd.Series(str(nextdt.date().strftime('%Y-%m-%d')) for _ in range(len(f))) #add a column called bhavcopydate and then fill it with the bhavecopy date
      f = f[['SYMBOL', 'BHAVCOPYDATE', 'OPEN', 'HIGH', 'LOW', 'CLOSE', 'TOTTRDQTY', 'DELIVERABLE','%DELIVERABLE']]
      f.to_csv(base+y+'/'+str(nextdt.date())+'.csv', index=False)
      os.remove(base+y+'/cm'+d+dmonth[m]+y+'bhav.csv')
      print('Merging of deliverable column to EQ bhavcopy...SUCCESS')
      
      ######################################################################################
      # connect to MySQL db in https://johnny.heliohost.org:2083/ UN 
      # pip install pip install mysql-connector --target=$nb_path pip install mysql-connector
      # https://pynative.com/python-mysql-database-connection/
      d, m, y = '%02d' % nextdt.day, '%02d' % nextdt.month, '%02d' % nextdt.year
      #check if the path exist and connect to cloud mysql 
      #if os.path.exists(base+y):
      try:
        connection = mysql.connector.connect(host='johnny.heliohost.org',
                                            database='akini_algotrade',
                                            user='akini',
                                            password='Drink7up@home')
        
        db_Info = connection.get_server_info()
        print("\nConnected to MySQL Server - version", db_Info)
        cursor = connection.cursor()
        cursor.execute("select database();")
        record = cursor.fetchone()
        print("\nIngesting EQUITIES bhavcopy "+'/'+y+'-'+m+'-'+d+'.csv'+" into DB....:", record)
        cursor.fast_executemany = True
        with open(base+y+'/'+y+'-'+m+'-'+d+'.csv', newline='',  encoding="utf8") as csvfile:
          csvdata = csv.reader(csvfile)
          #skip the 1st row as it will be header
          next(csvdata)
          pbar = ProgressBar()
          for row in pbar(list(csvdata)):
            # Prepare SQL query to INSERT a record into the database.
            sql_stocks = "INSERT INTO bhavcopy (symbol, bhavcopydate, open, high, low, close, tottrdqty, deliverable, deliverable_percent) \
            VALUES ('%s', '%s','%s', '%s','%s', '%s', '%s', '%s','%s');" % (row[0], row[1], row[2], row[3], row[4], row[5], row[6], row[7], row[8])
            #print(sql)
            try:
              #Execute the SQL command
              cursor.execute(sql_stocks)
              #Commit your changes in the database
              connection.commit()
            except Error as e:
              cprint('-------------------------------','red')
              print("Error while connecting to MySQL", e)
              cprint('-------------------------------','red')
              connection.rollback()
              #pbar.next()
              pbar.finish()
              break
      except Error as e:
        cprint('-------------------------------','red')
        print("Error while connecting to MySQL", e)
        cprint('-------------------------------','red')
        break
      finally:
        if (connection.is_connected()):
          cursor.close()
          connection.close()
    #now lets work with the FOREX
    for i in range(2): #try to connect to the nseindia url to download the bhavcopy , 3 times , just incase website does not respond etc.
      #while True:
      try:
        headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/68.0.3440.106 Safari/537.36',}#Sometimes requests from requests.get() gets blocked by server, so solution is to make the server think the request is coming from a web browser
        forex = requests.get('https://archives.nseindia.com/archives/cd/mkt_act/cd'+d+m+y+'.zip', headers=headers, timeout=5) #get the zip file from nseindia
      except requests.exceptions.Timeout as error:
        cprint('ERROR!!! 404 filenotfound cd'+d+m+y+'.zip..skipping','red')
        break
      except requests.ConnectionError:
        cprint('No connection.. retrying','red')
        break
    if forex.status_code==200:
        fx=open(zpath, 'wb') #open the zip file under a temp location called zpath that is defined above
        fx.write(forex.content)
        fx.close()
        z, wr = zipfile.ZipFile(zpath,'r'), nextdt.date()
        z.extractall(base+y+'/Forex') #extract the contents of the zip file to a location called forex , it extracts some 6 csv files we want only 1
        z.close()
        os.remove(zpath) #remove the zip file
        if os.path.exists(base+y+'/Forex/cf'+d+m+y+'.csv'):
          def trim(dataset): #Definition for strippping whitespace
              trim = lambda x: x.strip() if type(x) is str else x
              return dataset.applymap(trim)
          # making dataframe from csv file
          data = trim(pd.read_csv(base+y+'/Forex/cf'+d+m+y+'.csv'))  #reading the raw dl-ed file and trimming the trailing spaces with trim()
          data = data[data['INSTRUMENT'] == 'FUTCUR'] #retaining only FUTCUR rows and leaving out other rows
          data[['INSTRUMENT', 'SYMBOL    ', 'EXP_DATE  ', 'OPEN_PRICE ', 'HI_PRICE   ', 'LO_PRICE   ', 'CLOSE_PRICE','OPEN_INT*      ','TRD_VAL           ','TRD_QTY          ','NO_OF_CONT       ','NO_OF_TRADE      ']]#the useless file has spaces in the heading.
          data = data.rename(columns={'SYMBOL    ':'SYMBOL', 'EXP_DATE  ':'EXP_DATE', 'OPEN_PRICE ':'OPEN', 'HI_PRICE   ':'HIGH', 'LO_PRICE   ':'LOW', 'CLOSE_PRICE':'CLOSE','OPEN_INT*      ':'OPEN_INT','TRD_VAL           ':'TRD_VAL','TRD_QTY          ':'TRD_QTY','NO_OF_CONT       ':'NO_OF_CONT','NO_OF_TRADE      ':'NO_OF_TRADE'}) #rename some of the columns to something that is easier to underastand
          data['BHAVCOPYDATE'] = pd.Series(str(nextdt.date().strftime('%Y-%m-%d')) for _ in range(len(data))) #add a column called bhavcopydate and then fill it with the bhavecopy date
          data['EXP_DATE'] = pd.to_datetime(data['EXP_DATE'], format = '%d/%m/%Y') #the exp_date column is in format dd/mm/yyy convert it to yyyy-mm-dd
          data.to_csv(base+y+'/Forex/'+ str(nextdt.date())+'_forex.csv', index=False) #write the changes above to a new file and add an _forex to the csv file
          os.remove(base+y+'/Forex/cf'+d+m+y+'.csv')
          print('Forex bhavcopy '+ str(nextdt.date())+'_forex.csv create...SUCCESS')
          #print(data)
          ######################################################################################
          #cleanup all the useless files that get extracted from the currency futures bhavcopy
          for CleanUp in glob.glob(base+y+'/Forex/*.*'): # list out the files
            if not re.match(".+forex+",CleanUp): #if the list found above contains "forex" then dont do anything, else delete
              os.remove(CleanUp) #remove files in the folder
            
          try:
            connection = mysql.connector.connect(host='johnny.heliohost.org',
                                                database='akini_algotrade',
                                                user='akini',
                                                password='Drink7up@home')
            
            db_Info = connection.get_server_info()
            print("\nConnected to MySQL Server version", db_Info)
            #print(">>>>>>>>>>>>>>>\n")
            cursor = connection.cursor()
            cursor.execute("select database();")
            record = cursor.fetchone()
            print("\nIngesting FOREX bhavcopy "+'/'+y+'-'+m+'-'+d+'_forex.csv'+" into DB....:", record)
            #print(">>>>>>>>>>>>>>>\n")
            cursor.fast_executemany = True
            with open(base+y+'/Forex/'+ str(nextdt.date())+'_forex.csv', newline='',  encoding="utf8") as csvfile_forex:
              csvdata_forex = csv.reader(csvfile_forex)
              #skip the 1st row as it will be header
              next(csvdata_forex)
              pbar = ProgressBar()
              for row in pbar(list(csvdata_forex)):
              #Prepare SQL query to INSERT a record into the database.
                sql_forex = "INSERT INTO forex (symbol, exp_date, open, high, low, close, open_int, trd_val, trd_qty, no_of_cont, no_of_trade, bhavcopydate) \
                VALUES ('%s','%s','%s', '%s','%s', '%s', '%s', '%s','%s','%s', '%s','%s');" % (row[1], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[9], row[10], row[11], row[12])
                try:
                  #Execute the SQL command
                  cursor.execute(sql_forex)
                  #Commit your changes in the database
                  connection.commit()
                except Error as e:
                  cprint('-------------------------------','red')
                  print("Error while connecting to MySQL", e)
                  cprint('-------------------------------','red')
                  connection.rollback()
                  #pbar.next()
                  pbar.finish()
                  break
          except Error as e:
            cprint('-------------------------------','red')
            print("Error while connecting to MySQL", e)
            cprint('-------------------------------','red')
            break
          finally:
            if (connection.is_connected()):
              cursor.close()
              connection.close()
cprint(">>>>>>>>>>>>>>>>>>>>>>>>>>>>>\nDONE - All imports complete\n>>>>>>>>>>>>>>>>>>>>>>>>>>>>>",'green')

#sometimes nse doesnt give the index file, so the if condition
#        indices = requests.get('https://archives.nseindia.com/content/indices/ind_close_all_'+d+m+y+'.csv').content
#        if len(indices)>300:
#          indx = pd.read_csv(io.StringIO(indices.decode('utf-8'))) #reading content of indices csv and storing in DataFrame using io module
#          indx.to_csv(base+y+'/Index/Indices'+ str(nextdt.date())+'.csv', index=False)
#          indx[['Index Name', 'Index Date', 'Open Index Value', 'High Index Value', 'Low Index Value', 'Closing Index Value', 'Volume']]
#          indx = indx.rename(columns={'Index Name' : 'SYMBOL', 'Index Date' : 'BHAVCOPYDATE', 'Open Index Value' : 'OPEN', 'High Index Value' : 'HIGH', 'Low Index Value' : 'LOW', 'Closing Index Value' : 'CLOSE', 'Volume' : 'TOTTRDQTY'})
#          print('Indices bhavcopy file ind_close_all_'+d+m+y+'.csv downloaded successfully')
        #print(indx)
      ######################################################################################
      #futures download the zip from nseindia , and extract not inserting into db yet      #
      ######################################################################################
#        for i in range(7): #try to connect to the nseindia url to download the bhavcopy , 7 times , just incase website does not respond etc.
#          while True:
#            try:
#              futures = requests.get('https://archives.nseindia.com/content/historical/DERIVATIVES/'+y+'/'+dmonth[m]+'/fo'+d+dmonth[m]+y+'bhav.csv.zip')
#            except requests.ConnectionError:
#              print('No connection, retrying')
#              break
#        fo = open(zpath, 'wb')
#        fo.write(futures.content)
#        fo.close()
#        z, wr = zipfile.ZipFile(zpath,'r'), nextdt.date()
#        z.extractall(base+y+'/Futures')
#        z.close()
#        os.remove(zpath)
#       print('Futures bhavcopy file fo'+d+dmonth[m]+y+'bhav.csv downloaded successfully')

In [ ]:
import pandas as pd
import requests as requests
from progressbar import ProgressBar
import mysql.connector
from mysql.connector import Error

# Connect to the database
connection = mysql.connector.connect(host='johnny.heliohost.org',
                                                database='akini_algotrade',
                                                user='akini',
                                                password='Drink7up@home')
# create cursor
cursor=connection.cursor()
csvfile = requests.get('https://docs.google.com/spreadsheets/d/1ZTyh6GiHTwA1d-ApYdn5iCmRiBLZoAtwigS7VyLUk_Y/edit#gid=0')
print(csvfile.status_code)
df=pd.read_csv('https://docs.google.com/spreadsheets/d/1ZTyh6GiHTwA1d-ApYdn5iCmRiBLZoAtwigS7VyLUk_Y/export?format=csv&gid=0', usecols=["Stocks not allowed for MIS"])
# Insert DataFrame recrds one by one.
pbar = ProgressBar()
for i,row in pbar(df.iterrows()):
#Prepare SQL query to INSERT a record into the database.
  sql_banned_symbols = "INSERT INTO banned_symbols (symbol) VALUES ('%s');" % (row[0])
  try:
    #Execute the SQL command
    cursor.execute(sql_banned_symbols)
    #Commit your changes in the database
    connection.commit()
  except Error as e:
    cprint('-------------------------------','red')
    print("Error while connecting to MySQL", e)
    cprint('-------------------------------','red')
    connection.rollback()
    #pbar.next()
    pbar.finish()
    break
if (connection.is_connected()):
              cursor.close()
              connection.close()
df

# most volatile stocks in the database in the last 1 week
SELECT symbol,CLOSE,`bhavcopydate`, VARIANCE(high-low) var 
  FROM bhavcopy 
  where `tottrdqty`> '10000000' and CLOSE between 100 and 200
  and `bhavcopydate` > curdate()-20
  GROUP BY symbol  
ORDER BY `var`  DESC

#stocks that are most standard "low-high" movements , good for predicting possible upside left in a stock during intraday%
SELECT symbol, high, low, close, bhavcopydate, stddev(high-low) 'stddev - the closer to 0 better it is'
  FROM bhavcopy 
  where bhavcopydate >curdate()-10
  and close between 100 and 200
  and tottrdqty >10000000
  group by symbol
  order by 'stddev - the closer to 0 better it is'

# most upside or downside potential of a stock
SELECT AVG(uspot_array.uspot) as median_upside_potential
FROM (
SELECT ((high-open)/open)*100 as uspot, @rownum:=@rownum+1 as `row_number`, @total_rows:=@rownum
  FROM bhavcopy , (SELECT @rownum:=0) r
  WHERE symbol='granules'
  ORDER BY uspot
) as uspot_array
WHERE uspot_array.row_number IN ( FLOOR((@total_rows+1)/2), FLOOR((@total_rows+2)/2) );

SELECT AVG(dspot_array.dspot) as median_downside_potential
FROM (
SELECT ((low-open)/open)*100 as dspot, @rownum:=@rownum+1 as `row_number`, @total_rows:=@rownum
  FROM bhavcopy , (SELECT @rownum:=0) r
  WHERE symbol='granules'
  ORDER BY dspot
) as dspot_array
WHERE dspot_array.row_number IN ( FLOOR((@total_rows+1)/2), FLOOR((@total_rows+2)/2) );


# most upside or downside potential of a currency future
SELECT AVG(uspot_array.uspot) as median_upside_potential
FROM (
SELECT ((high-open)/open)*100 as uspot, @rownum:=@rownum+1 as `row_number`, @total_rows:=@rownum
  FROM forex , (SELECT @rownum:=0) r
  WHERE symbol='USDINR' and exp_date='2020-07-29'
  ORDER BY uspot
) as uspot_array
WHERE uspot_array.row_number IN ( FLOOR((@total_rows+1)/2), FLOOR((@total_rows+2)/2) );

SELECT AVG(dspot_array.dspot) as median_downside_potential
FROM (
SELECT ((low-open)/open)*100 as dspot, @rownum:=@rownum+1 as `row_number`, @total_rows:=@rownum
  FROM forex , (SELECT @rownum:=0) r
  WHERE symbol='USDINR' and exp_date='2020-07-29'
  ORDER BY dspot
) as dspot_array
WHERE dspot_array.row_number IN ( FLOOR((@total_rows+1)/2), FLOOR((@total_rows+2)/2) );

In [ ]:
import os, sys
from google.colab import drive
drive.mount('/content/drive')
nb_path = '/content/notebooks'
os.symlink('/content/drive/My Drive/Colab Notebooks', nb_path)
sys.path.insert(0, nb_path)  # or append(nb_path)